In [1]:
dbutils.library.installPyPI("mlflow", "1.0.0")

Out[1]: True

In [2]:
import boto3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import mlflow.sklearn

Sat Mar 28 02:46:56 2020 py4j imported

# 1. Select any of the F1 datasets in AWS S3 to build your model. You are allowed to join multiple datasets.
Will build a model to predict pit times using the pit_stops dataset and the races dataset (joined to get circuitId to use as a feature). Data imported and joined below.

In [4]:
s3 = boto3.client(
    's3',
    aws_access_key_id='',
    aws_secret_access_key=''
)

In [5]:
bucket = "ne-gr5069"
f1_pit_stops = "raw/pit_stops.csv"
f1_races = "raw/races.csv"

obj_pit_stops = s3.get_object(Bucket= bucket, Key= f1_pit_stops) 
obj_races = s3.get_object(Bucket= bucket, Key= f1_races) 

In [6]:
df_pit_stops = pd.read_csv(obj_pit_stops['Body'])
display(df_pit_stops)

raceId,driverId,stop,lap,time,duration,milliseconds
841,153,1,1,17:05:23,26.898,26898
841,30,1,1,17:05:52,25.021,25021
841,17,1,11,17:20:48,23.426,23426
841,4,1,12,17:22:34,23.251,23251
841,13,1,13,17:24:10,23.842,23842
841,22,1,13,17:24:29,23.643,23643
841,20,1,14,17:25:17,22.603,22603
841,814,1,14,17:26:03,24.863,24863
841,816,1,14,17:26:50,25.259,25259
841,67,1,15,17:27:34,25.342,25342


In [7]:
df_races = pd.read_csv(obj_races['Body'])
df_races_select = df_races[['raceId', 'circuitId', 'name']]
display(df_races_select)

raceId,circuitId,name
1,1,Australian Grand Prix
2,2,Malaysian Grand Prix
3,17,Chinese Grand Prix
4,3,Bahrain Grand Prix
5,4,Spanish Grand Prix
6,6,Monaco Grand Prix
7,5,Turkish Grand Prix
8,9,British Grand Prix
9,20,German Grand Prix
10,11,Hungarian Grand Prix


In [8]:
df_pits_races = df_pit_stops.merge(df_races_select, on = 'raceId', how = 'left')

display(df_pits_races)

raceId,driverId,stop,lap,time,duration,milliseconds,circuitId,name
841,153,1,1,17:05:23,26.898,26898,1,Australian Grand Prix
841,30,1,1,17:05:52,25.021,25021,1,Australian Grand Prix
841,17,1,11,17:20:48,23.426,23426,1,Australian Grand Prix
841,4,1,12,17:22:34,23.251,23251,1,Australian Grand Prix
841,13,1,13,17:24:10,23.842,23842,1,Australian Grand Prix
841,22,1,13,17:24:29,23.643,23643,1,Australian Grand Prix
841,20,1,14,17:25:17,22.603,22603,1,Australian Grand Prix
841,814,1,14,17:26:03,24.863,24863,1,Australian Grand Prix
841,816,1,14,17:26:50,25.259,25259,1,Australian Grand Prix
841,67,1,15,17:27:34,25.342,25342,1,Australian Grand Prix


# 2. Build any model of your choice
Use df_pits_races to build a random forest model...

**Features**
* Circuit ID (OneHotEncoded)
* driverId (OneHotEncoded)
* stop
* lap

**Target**
* Milliseconds

In [10]:
df_pits_races_4_model = df_pits_races[['circuitId', 
                                      'driverId',
                                      'stop',
                                      'lap',
                                      'milliseconds']]

df_pits_races_4_model_encoded = pd.get_dummies(df_pits_races_4_model,
                                              columns = ['circuitId',
                                                         'driverId']
                                              )

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df_pits_races_4_model_encoded.drop(["milliseconds"],
                                                                                       axis=1),
                                                    df_pits_races_4_model_encoded[["milliseconds"]],
                                                    random_state=42)

# ML Flow Setup
### 3. Log the parameters used in the model in each run
### 4. Log the model
### 5. Log every possible metric from the model
### 6. Log at least two artifacts (plots, or csv files)

Logged artifacts - residual plot and feature importance CSV is available in the
detailed run pages. For run 12 (the best model) These artifacts have been
downloaded and pushed to the repo in the folder deliverables/run_12_logged_items

In [13]:
with mlflow.start_run(run_name="Basic Experiment") as run:
  runID = run.info.run_uuid
  experimentID = run.info.experiment_id

In [14]:
def log_rf(experimentID, run_name, params, X_train, X_test, y_train, y_test):
  import os
  import matplotlib.pyplot as plt
  import seaborn as sns
  from sklearn.ensemble import RandomForestRegressor
  from sklearn.metrics import explained_variance_score, max_error
  from sklearn.metrics import mean_absolute_error, mean_squared_error
  from sklearn.metrics import mean_squared_log_error, median_absolute_error 
  from sklearn.metrics import r2_score, mean_poisson_deviance
  from sklearn.metrics import mean_gamma_deviance
  import tempfile

  with mlflow.start_run(experiment_id=experimentID, run_name=run_name) as run:
    # Create model, train it, and create predictions
    rf = RandomForestRegressor(**params)
    rf.fit(X_train, y_train)
    predictions = rf.predict(X_test)

    # Log model
    mlflow.sklearn.log_model(rf, "random-forest-model")

    # Log params
    [mlflow.log_param(param, value) for param, value in params.items()]

    # Create metrics
    exp_var = explained_variance_score(y_test, predictions)
    max_err = max_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = mean_squared_error(y_test, predictions, squared = False)
    mslogerror = mean_squared_log_error(y_test, predictions)
    medianae = median_absolute_error(y_test,predictions)
    r2 = r2_score(y_test, predictions)
    mean_poisson = mean_poisson_deviance(y_test, predictions)
    mean_gamma = mean_gamma_deviance(y_test, predictions)
    
    # Print metrics
    print("  explained variance: {}".format(exp_var))
    print("  max error: {}".format(max_err))
    print("  mae: {}".format(mae))
    print("  mse: {}".format(mse))
    print("  rmse: {}".format(rmse))
    print("  mean square log error: {}".format(mslogerror))
    print("  median abosulte error: {}".format(medianae))
    print("  R2: {}".format(r2))
    print("  mean poisson deviance: {}".format(mean_poisson))    
    print("  mean gamma deviance: {}".format(mean_gamma))
    
    # Log metrics
    mlflow.log_metric("explained variance", exp_var)
    mlflow.log_metric("max error", max_err)  
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)  
    mlflow.log_metric("mean square log error", mslogerror)  
    mlflow.log_metric("median abosulte error", medianae)
    mlflow.log_metric("R2", r2)  
    mlflow.log_metric("mean poisson deviance", mean_poisson)  
    mlflow.log_metric("mean gamma deviance", mean_gamma)

    
    # Create feature importance
    importance = pd.DataFrame(list(zip(df_pits_races_4_model_encoded.columns,
                                       rf.feature_importances_)), 
                                columns=["Feature", "Importance"]
                              ).sort_values("Importance", ascending=False)
    
    # Log importances using a temporary file
    temp = tempfile.NamedTemporaryFile(prefix="feature-importance-", suffix=".csv")
    temp_name = temp.name
    try:
      importance.to_csv(temp_name, index=False)
      mlflow.log_artifact(temp_name, "feature-importance.csv")
    finally:
      temp.close() # Delete the temp file
    
    # Create plot
    fig, ax = plt.subplots()

    sns.residplot(predictions, y_test.values.ravel(), lowess=False)
    plt.xlabel("Predicted values pit duration")
    plt.ylabel("Residual")
    plt.title("Residual Plot for pitting")

    # Log residuals using a temporary file
    temp = tempfile.NamedTemporaryFile(prefix="residuals_pit_model", suffix=".png")
    temp_name = temp.name
    try:
      fig.savefig(temp_name)
      mlflow.log_artifact(temp_name, "residuals_pit_model.png")
    finally:
      temp.close() # Delete the temp file
      
    display(fig)
    return run.info.run_uuid

# Model training and tracking
### 7. Track your MLFlow experiment and run at least 10 with different parameters

In [16]:
params_run1 = {'n_estimators': 100,
               'max_depth': 5,
               'random_state': 42
              }

log_rf(experimentID, 'Run 1', params_run1, X_train, X_test, y_train, y_test)

In [17]:
params_run2 = {'n_estimators': 100,
               'max_depth': 4,
               'random_state': 42
              }

log_rf(experimentID, 'Run 2', params_run2, X_train, X_test, y_train, y_test)

In [18]:
params_run3 = {'n_estimators': 100,
               'max_depth': 3,
               'random_state': 42
              }

log_rf(experimentID, 'Run 3', params_run3, X_train, X_test, y_train, y_test)

In [19]:
params_run4 = {'n_estimators': 100,
               'max_depth': 2,
               'random_state': 42
              }

log_rf(experimentID, 'Run 4', params_run4, X_train, X_test, y_train, y_test)

In [20]:
params_run5 = {'n_estimators': 100,
               'max_depth': 1,
               'random_state': 42
              }

log_rf(experimentID, 'Run 5', params_run5, X_train, X_test, y_train, y_test)

In [21]:
params_run6 = {'n_estimators': 1000,
               'max_depth': 5,
               'random_state': 42
              }

log_rf(experimentID, 'Run 6', params_run6, X_train, X_test, y_train, y_test)

In [22]:
params_run7 = {'n_estimators': 1000,
               'max_depth': 4,
               'random_state': 42
              }

log_rf(experimentID, 'Run 7', params_run7, X_train, X_test, y_train, y_test)

In [23]:
params_run8 = {'n_estimators': 1000,
               'max_depth': 3,
               'random_state': 42
              }

log_rf(experimentID, 'Run 8', params_run8, X_train, X_test, y_train, y_test)

In [24]:
params_run9 = {'n_estimators': 1000,
               'max_depth': 2,
               'random_state': 42
              }

log_rf(experimentID, 'Run 9', params_run9, X_train, X_test, y_train, y_test)

In [25]:
params_run10 = {'n_estimators': 1000,
               'max_depth': 1,
               'random_state': 42
              }

log_rf(experimentID, 'Run 10', params_run10, X_train, X_test, y_train, y_test)

In [26]:
params_run11 = {'n_estimators': 100,
               'max_depth': 6,
               'random_state': 42
              }

log_rf(experimentID, 'Run 11', params_run11, X_train, X_test, y_train, y_test)

In [27]:
params_run12 = {'n_estimators': 1000,
               'max_depth': 6,
               'random_state': 42
              }

log_rf(experimentID, 'Run 12', params_run12, X_train, X_test, y_train, y_test)

# 8. Select your best model run and explain why

Run 12 is the best model run. It has the best score between all models that were tested
on multiple metrics. Actually, if I understand mean gamma deviance and mean poisson 
deviance correctly, it does the best on all the available metrics sklearn has for 
regression (except median absolute error where it loses to run 11), so there isn't
much contest.

You can also see from ther residual plots that while all the RF models tend to 
mis-predict in similar ways, by looking at the y-axis you can see that mispredictions
are kept to a smaller (absolute) range when max_depth is higher (with it being highest
for runs 11 and 12). Comparing just run 11 and 12 (which differ in the number of trees),
you can't see much difference, but the metrics tip the scales in favour of run 12 (which
didn't take much longer to train than run 11 anyway, despite having 10x more trees).

### 9. Take a screenshot of your MLFlow Homepage as part of your assignment submission 
### 10. Take a screenshot of your detailed run page

Both of MLFlow Homepage screenshot and detailed run page for Run 12 have been
added to the repo in the folder deliverables/mlflow_screenshots